In [ ]:
# lr, size, range, algo

import torch, random
import torch.nn as nn
import torch.optim as optim


class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 6)
        self.fc2 = nn.Linear(6, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

class SeqModel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
      super(SeqModel, self).__init__()
      self.linear_layer_stack = nn.Sequential(
            nn.Linear(input_features, hidden_units),
            nn.ReLU(True),
            nn.Linear(hidden_units, output_features),
        )

    def forward(self, x):
      return self.linear_layer_stack(x)


In [ ]:
#X_train = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
#y_train = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

X_trn = [[random.uniform(0, 1), random.uniform(0, 1)] for _ in range(3000)]
y_trn = [[round(X_trn[i][0])^round(X_trn[i][1])] for i in range(3000)]
X_train = torch.tensor(X_trn, dtype=torch.float32)
y_train = torch.tensor(y_trn, dtype=torch.float32)
print(X_train[:10], y_train[:10])

# Instantiate the Model, Define Loss Function and Optimizer
model = SimpleNN()
#criterion = nn.MSELoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

for epoch in range(500):
    model.train()

    # Forward pass
    outputs = model(X_train)
    #print(outputs)
    loss = criterion(outputs, y_train)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/500], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    test_data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
    predictions = model(test_data)
    print(f'Predictions:\n{predictions}')
    # results very sensitive to random initial parameters


# Instantiate the Model, Define Loss Function and Optimizer
model_seq = SeqModel(2,1)
#criterion_seq = nn.MSELoss()
criterion_seq = nn.BCEWithLogitsLoss()
optimizer_seq = optim.Adam(model_seq.parameters(), lr=0.01)

for epoch in range(500):
    model_seq.train()

    # Forward pass
    outputs_seq = model_seq(X_train)
    loss_seq = criterion_seq(outputs_seq, y_train)

    # Backward pass and optimize
    optimizer_seq.zero_grad()
    loss_seq.backward()
    optimizer_seq.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/500], Loss: {loss_seq.item():.4f}')

model_seq.eval()
with torch.no_grad():
    test_data_seq = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
    predictions_seq = model_seq(test_data_seq)
    print(f'Predictions:\n{predictions_seq}')
    # results very sensitive to random initial parameters

tensor([[0.2004, 0.9548],
        [0.2228, 0.6909],
        [0.0244, 0.2827],
        [0.8088, 0.9146],
        [0.1571, 0.5011],
        [0.8398, 0.3143],
        [0.5947, 0.1814],
        [0.3845, 0.4351],
        [0.7878, 0.8607],
        [0.3123, 0.9838]]) tensor([[1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.]])
Epoch [100/500], Loss: 0.6691
Epoch [200/500], Loss: 0.5865
Epoch [300/500], Loss: 0.5519
Epoch [400/500], Loss: 0.5403
Epoch [500/500], Loss: 0.5344
Predictions:
tensor([[7.7946e-14],
        [9.5812e-01],
        [9.9998e-01],
        [1.4803e-13]])
Epoch [100/500], Loss: 0.5811
Epoch [200/500], Loss: 0.3990
Epoch [300/500], Loss: 0.3467
Epoch [400/500], Loss: 0.3456
Epoch [500/500], Loss: 0.3456
Predictions:
tensor([[-3.7573],
        [ 6.9890],
        [ 8.3800],
        [-3.5726]])


In [ ]:
print(model(torch.tensor([[0.51,0.01]])))

tensor([[0.9962]], grad_fn=<SigmoidBackward0>)


In [ ]:
class Generator(nn.Module):
    def __init__(self, noise_dim):
        super(Generator, self).__init__()
        self.noise_dim = noise_dim
        self.fc1 = nn.Linear(noise_dim, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Discriminator(nn.Module):
  def __init__(self):
      super(Discriminator, self).__init__()
      self.fc1 = nn.Linear(2, 6)
      self.fc2 = nn.Linear(6, 4)
      self.fc3 = nn.Linear(4, 1)
      self.relu = nn.ReLU()
      self.sigmoid = nn.Sigmoid()

  def forward(self, x):
      x = self.relu(self.fc1(x))
      x = self.relu(self.fc2(x))
      x = self.fc3(x)
      return x

In [ ]:
NOISE_DIM = 4

generator = Generator(NOISE_DIM)
discriminator = Discriminator()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = generator.to(device)
discriminator = discriminator.to(device)

criterion = nn.BCEWithLogitsLoss()
#criterion = nn.MSELoss()

generator_optimizer = optim.Adam(generator.parameters(), lr=0.02, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.02, betas=(0.5, 0.999))

NUM_EPOCHS = 200
BATCH_SIZE = 256
SAMPLE_SIZE = 10000

x_trn = [[random.uniform(0, 1), random.uniform(0, 1)] for _ in range(SAMPLE_SIZE)]
y_trn = [round(x_trn[i][0])^round(x_trn[i][1]) for i in range(SAMPLE_SIZE)]
train_dataset =  [[x_trn[i][0],x_trn[i][1],y_trn[i]] for i in range(SAMPLE_SIZE)]
train_dataset = torch.tensor(train_dataset)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# malfunctioning, attempting ideas: discriminator~basic dnn with gan attacked

for epoch in range(NUM_EPOCHS):
    for test in train_loader:

      real_images = test[:,:2]
      #real_images = real_images.to(device)

      discriminator_optimizer.zero_grad()
      #real_labels = torch.ones(real_images.size(0), 1, device=device)
      real_labels = test[:,2:]
      #print(real_labels)
      real_outputs = discriminator(real_images)
      #print(real_outputs)
      #print(real_labels)
      real_loss = criterion(real_outputs, real_labels)
      #real_loss.backward()

      noise = torch.randn(real_images.size(0), NOISE_DIM, device=device)
      #print(noise)
      fake_images = generator(noise)
      real_list = real_labels.tolist()
      fake_labels = torch.tensor([[round(real_list[i][0])^1] for i in range(len(real_list))], dtype=torch.float32)
      fake_outputs = discriminator(fake_images.detach())
      #print(fake_labels)

      #fo_tl = fake_outputs.tolist()
      #fake_outputs_labels = torch.tensor([[round(fo_tl[i][0])] for i in range(len(fo_tl))], dtype=torch.float32)
      fake_outputs_labels = torch.zeros(real_images.size(0), 1, device=device)
      fake_loss = criterion(fake_outputs, fake_outputs_labels)
      #print(fake_outputs_labels)
      #fake_loss.backward()
      d_loss = (real_loss + fake_loss) / 2
      d_loss.backward()
      discriminator_optimizer.step()

      generator_optimizer.zero_grad()
      #fake_labels = real_labels
      gen_images = generator(noise)
      gen_outputs = discriminator(gen_images)
      gen_loss = criterion(gen_outputs, real_labels)
      gen_loss.backward()
      generator_optimizer.step()

    if epoch == 0 or epoch == NUM_EPOCHS-1:
      print(f'Epoch [{epoch+1}/{NUM_EPOCHS}]'#, Step [{epoch+1}/{len(train_loader)}], '
            f'Discriminator Loss: {d_loss.item():.4f}, '
            f'Generator Loss: {gen_loss.item():.4f}')
    elif epoch % 10 == 0 and epoch != 0:
      print(f'Epoch [{epoch}/{NUM_EPOCHS}]'#, Step [{epoch+1}/{len(train_loader)}], '
            f'Discriminator Loss: {d_loss.item():.4f}, '
            f'Generator Loss: {gen_loss.item():.4f}')

Epoch [1/200]Discriminator Loss: 0.4804, Generator Loss: 1.0834
Epoch [10/200]Discriminator Loss: 0.3128, Generator Loss: 1.0987
Epoch [20/200]Discriminator Loss: 0.3717, Generator Loss: 1.0267
Epoch [30/200]Discriminator Loss: 0.4158, Generator Loss: 1.2639
Epoch [40/200]Discriminator Loss: 0.3635, Generator Loss: 1.0886
Epoch [50/200]Discriminator Loss: 0.2084, Generator Loss: 0.6750
Epoch [60/200]Discriminator Loss: 0.5735, Generator Loss: 1.2487
Epoch [70/200]Discriminator Loss: 0.3346, Generator Loss: 1.1075
Epoch [80/200]Discriminator Loss: 0.2101, Generator Loss: 0.8472
Epoch [90/200]Discriminator Loss: 0.2119, Generator Loss: 0.6646
Epoch [100/200]Discriminator Loss: 0.2617, Generator Loss: 1.0231
Epoch [110/200]Discriminator Loss: 0.0063, Generator Loss: 2.3623
Epoch [120/200]Discriminator Loss: 0.0228, Generator Loss: 3.7128
Epoch [130/200]Discriminator Loss: 0.0008, Generator Loss: 2.4217
Epoch [140/200]Discriminator Loss: 0.0010, Generator Loss: 1.7247
Epoch [150/200]Discri

In [ ]:
def generate_and_save_images(model, dis_model, epoch, noise):
    model.eval()
    with torch.no_grad():
        fake_images = model(noise).cpu()
        print(noise)
        print(fake_images)
        print(dis_model(fake_images))
test_noise = torch.randn(4, NOISE_DIM, device=device)
generate_and_save_images(generator, discriminator, NUM_EPOCHS, test_noise)

tensor([[ 0.6106,  0.0590, -0.5936, -1.9090],
        [ 0.4375, -0.3661,  1.8641,  0.7534],
        [-0.9431,  0.2412,  2.6971,  0.8192],
        [-1.0979, -0.2990,  0.7012, -0.5589]])
tensor([[-0.9844,  0.2292],
        [-0.9844,  0.2292],
        [-0.9844,  0.2292],
        [-0.9844,  0.2292]])
tensor([[-8.2689],
        [-8.2689],
        [-8.2689],
        [-8.2689]])


In [ ]:
print(generator(torch.randn(1, NOISE_DIM, device=device)))
print(discriminator(torch.tensor([[0.4901,0.6013]])))

tensor([[-0.9844,  0.2292]], grad_fn=<AddmmBackward0>)
tensor([[3.2466]], grad_fn=<AddmmBackward0>)


In [ ]:
class Generator_Thre(nn.Module):
    def __init__(self, noise_dim):
        super(Generator_Thre, self).__init__()
        self.noise_dim = noise_dim
        self.fc1 = nn.Linear(noise_dim, 2)
        self.fc2 = nn.Linear(2, 4)
        self.fc3 = nn.Linear(4, 2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Discriminator_Thre(nn.Module):
    def __init__(self):
        super(Discriminator_Thre, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 4)
        self.fc3 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
NOISE_DIM = 4

generator_thre = Generator_Thre(NOISE_DIM)
discriminator_thre = Discriminator_Thre()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator_thre = generator_thre.to(device)
discriminator_thre = discriminator_thre.to(device)

criterion_thre = nn.BCEWithLogitsLoss()
#criterion = nn.MSELoss()

generator_thre_optimizer = optim.Adam(generator_thre.parameters(), lr=0.02, betas=(0.5, 0.999))
discriminator_thre_optimizer = optim.Adam(discriminator_thre.parameters(), lr=0.02, betas=(0.5, 0.999))

NUM_EPOCHS = 200
BATCH_SIZE = 256
SAMPLE_SIZE = 10000

x_trn = [[random.uniform(0, 1), random.uniform(0, 1)] for _ in range(SAMPLE_SIZE)]
y_trn = [round(x_trn[i][0])^round(x_trn[i][1]) for i in range(SAMPLE_SIZE)]
train_dataset =  [[x_trn[i][0],x_trn[i][1],y_trn[i]] for i in range(SAMPLE_SIZE)]
train_dataset = torch.tensor(train_dataset)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# theoretical gan: simply generate images and making the discriminator considering fake as real

for epoch in range(NUM_EPOCHS):
    for test in train_loader:

      real_signal = test[:,:2]
      #real_images = real_images.to(device)

      real_labels = torch.ones(real_signal.size(0), 1, device=device)
      fake_labels = torch.zeros(real_signal.size(0), 1, device=device)

      discriminator_thre_optimizer.zero_grad()

      noise = torch.randn(real_signal.size(0), NOISE_DIM, device=device)

      #real_labels = test[:,2:]
      #print(real_labels)
      real_outputs = discriminator_thre(real_signal)
      real_loss = criterion_thre(real_outputs, real_labels)
      #real_loss.backward()

      #print(noise)
      fake_signal = generator_thre(noise)
      #real_list = real_labels.tolist()
      #fake_labels = torch.tensor([[round(real_list[i][0])^1] for i in range(len(real_list))], dtype=torch.float32)
      fake_outputs = discriminator_thre(fake_signal)
      #print(fake_labels)

      #fo_tl = fake_outputs.tolist()
      #fake_outputs_labels = torch.tensor([[round(fo_tl[i][0])] for i in range(len(fo_tl))], dtype=torch.float32)
      fake_loss = criterion_thre(fake_outputs, fake_labels)
      #print(fake_outputs_labels)
      #fake_loss.backward()
      d_loss = (real_loss + fake_loss) / 2
      d_loss.backward()
      discriminator_thre_optimizer.step()

      generator_thre_optimizer.zero_grad()
      gen_signal = generator_thre(noise)
      gen_outputs = discriminator_thre(gen_signal)
      gen_loss = criterion_thre(gen_outputs, real_labels)
      gen_loss.backward()
      generator_thre_optimizer.step()

    if epoch == 0 or epoch == NUM_EPOCHS-1:
      print(f'Epoch [{epoch+1}/{NUM_EPOCHS}]'#, Step [{epoch+1}/{len(train_loader)}], '
            f'Discriminator Loss: {d_loss.item():.4f}, '
            f'Generator Loss: {gen_loss.item():.4f}')
    elif epoch % 10 == 0 and epoch != 0:
      print(f'Epoch [{epoch}/{NUM_EPOCHS}]'#, Step [{epoch+1}/{len(train_loader)}], '
            f'Discriminator Loss: {d_loss.item():.4f}, '
            f'Generator Loss: {gen_loss.item():.4f}')

Epoch [1/200]Discriminator Loss: 0.7069, Generator Loss: 0.8532
Epoch [10/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [20/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [30/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [40/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [50/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [60/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [70/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [80/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [90/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [100/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [110/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [120/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [130/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [140/200]Discriminator Loss: 0.6931, Generator Loss: 0.6931
Epoch [150/200]Discri

In [ ]:
def generate_and_save_images(model, dis_model, epoch, noise):
    model.eval()
    with torch.no_grad():
        fake_images = model(noise).cpu()
        print(noise)
        print(fake_images)
        print(dis_model(fake_images))
test_noise = torch.randn(4, NOISE_DIM, device=device)
generate_and_save_images(generator_thre, discriminator_thre, NUM_EPOCHS, test_noise)

tensor([[-1.5961, -1.1986,  0.6461, -0.5769],
        [-0.3068, -0.1979, -0.2972,  1.4376],
        [-0.0141, -0.9585,  2.4865, -1.2544],
        [-0.5225, -1.2762,  1.6555, -0.5477]])
tensor([[-1.4096,  6.6109],
        [ 0.4984,  0.4962],
        [-2.7901,  5.9893],
        [-1.5081,  3.9546]])
tensor([[4.9303e-08],
        [4.9303e-08],
        [4.9303e-08],
        [4.9303e-08]])


In [ ]:
print(discriminator_thre(torch.tensor([[0.53,0.83]])))

tensor([[4.9303e-08]], grad_fn=<AddmmBackward0>)
